In [ ]:
import datetime, gc, random
import json, numpy as np, pandas as pd, zipfile
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.cluster import MeanShift, estimate_bandwidth

In [ ]:
# reading training data
path = "/kaggle/input/pkdd-15-predict-taxi-service-trajectory-i/"
zf = zipfile.ZipFile(path + 'train.csv.zip')
df = pd.read_csv(zf.open('train.csv'), converters={'POLYLINE': lambda x: json.loads(x)[-1:]})
latlong = np.array([[p[0][1], p[0][0]] for p in df['POLYLINE'] if len(p)>0])

In [ ]:
df.shape, latlong.shape

In [ ]:
df.head()

In [ ]:
bw = estimate_bandwidth(latlong, quantile=.1, n_samples=1000)
ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(latlong)
cluster_centers = ms.cluster_centers_
print("Clusters shape: ", cluster_centers.shape)

In [ ]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
# Plot result
plt.figure(1,figsize=(20,10))
plt.clf()
X = latlong
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

**Remove outliters**

In [ ]:
# Filter outliers
ll_df = pd.DataFrame(latlong, columns=['lat','long'])
ll_df.shape

In [ ]:
# Get data in Mean +- n*Std
n = 6
max_lat = latlong[:,0].mean() + n * latlong[:,0].std()
min_lat = latlong[:,0].mean() - n * latlong[:,0].std()
max_lon = latlong[:,1].mean() + n * latlong[:,1].std()
min_lon = latlong[:,1].mean() - n * latlong[:,1].std()
min_lat, max_lat, min_lon, max_lon

In [ ]:
cond1 = (ll_df.lat > min_lat) & (ll_df.lat < max_lat)
cond2 = (ll_df.long > min_lon) & (ll_df.long < max_lon)
ll_df = ll_df[cond1]
ll_df = ll_df[cond2]
ll_final = ll_df.values
latlong = ll_final

In [ ]:
# Clustering with Mean shift
# bw = estimate_bandwidth(latlong, quantile=.1, n_samples=1000)
bw = 0.001
ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(latlong)
cluster_centers = ms.cluster_centers_

print("Clusters shape: ", cluster_centers.shape)

In [ ]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
# Plot result
plt.figure(1,figsize=(20,10))
plt.clf()
X = latlong
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()